In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
import cv2

# Set the path to your dataset
train_dir = os.path.abspath('D:\\Human Dataset')
test_dir = os.path.abspath('D:\\Human Dataset')

# New resolution for face images
new_resolution = (150, 150)

# Number of classes in your dataset
num_classes = 10  # Adjust this to match the number of classes in your dataset

from keras.utils import to_categorical
train_generator.classes = to_categorical(train_generator.classes)

# Updated preprocess function
def preprocess_with_face_detection(image, target_size):
    # Convert the image to 8-bit unsigned integers
    image = cv2.convertScaleAbs(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        x, y, w, h = faces[n]  # Assuming there's only one face detected
        face = image[y:y+h, x:x+w]
        face_resized = cv2.resize(face, target_size)
        return face_resized / 255.0
    else:
        # Return an array of zeros if no face is detected
        return np.zeros(target_size + (3,)) / 255.0

# Updated data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=lambda img: preprocess_with_face_detection(img, new_resolution)
)
# Create the train and test generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=new_resolution,
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorcial'  # For classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=new_resolution,
    batch_size=32,  # Adjust batch size as needed
    class_mode='categorical'  # For binary classification
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    preprocessing_function=lambda img: preprocess_with_face_detection(img, new_resolution)
)

# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(new_resolution[0], new_resolution[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='sigmoid')  # Adjust activation function for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary_crossentropy for binary classification

# Print model summary
model.summary()
# Train the model

epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
# Define a list to store file paths and their corresponding predictions
predictions_with_paths = []

# During testing, for each image, get its file path and add it to the list
for test_image, test_label in test_generator:
    predictions = model.predict(test_image)
    for i in range(len(predictions)):
        # Get the predicted class label (replace this with your logic)
        predicted_label = np.argmax(predictions[i])

        # Get the original file path from the test_generator (assuming the test_generator is a DirectoryIterator)
        file_path = test_generator.filepaths[test_generator.batch_index - 1 + i]

        # Store the file path and predicted label in the list
        predictions_with_paths.append((file_path, predicted_label))

# Now you have a list of (file_path, predicted_label) pairs

# You can access this list to find file paths associated with similar images

# Example: Find file paths for images predicted as class 3 (adjust the class as needed)
similar_images_class_3 = [path for path, label in predictions_with_paths if label == 3]
print("File paths for similar images predicted as class 3:")
for path in similar_images_class_3:
    print(path)

ValueError: Invalid class_mode: categorial; expected one of: {'binary', 'categorical', 'sparse', 'input', None}